# Running GMM

### Imorting packages and data

In [ ]:
#%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import six
import tensorflow as tf
import os
import pandas as pd

from edward.models import Categorical

#from Models.GMM.Save import simple_save

from edward.models import (
    Categorical, Dirichlet, Empirical, InverseGamma,
    MultivariateNormalDiag, Normal, ParamMixture, RandomVariable, Bernoulli)
from tensorflow.contrib.distributions import Distribution

plt.style.use('ggplot')


###########
#For saving models:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.framework import ops
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants
#from tensorflow.python.util.tf_export import tf_export

#Reading data:
subset= pd.read_csv('\user_rec.csv', sep=",", header=0)
x_trainn=subset.values                                                  #Redefine x_train into an array instead of dataframe

#Standardizing:
from sklearn import preprocessing
X= preprocessing.scale(x_trainn)


#Defining test & train:
x_train= X[0:4984,]                                                  #Defining training data
x_test= X[4984:,]                                                   #Defining test data
N_testt= len(x_test)                                                    #Defining the length of the test-set


N = len(x_train)  # number of data points                               #Setting parameters - N is defined from the number of rows
K = 30  # number of components                                           #Setting parameters - number of clusters
D = x_train.shape[1]  # dimensionality of data                           #Setting parameters - dimension of data
ed.set_seed(42)


### Definerer modellen

In [ ]:
#Model:
pi = Dirichlet(tf.ones(K))                                              
                                                                        
mu = Normal(tf.zeros(D), tf.ones(D), sample_shape=K)                    
sigmasq = InverseGamma(tf.ones(D), tf.ones(D), sample_shape=K)          
                                                                        
x = ParamMixture(pi, {'loc': mu, 'scale_diag': tf.sqrt(sigmasq)},       
                 MultivariateNormalDiag,                                
                 sample_shape=N)
z = x.cat                                                               


#Inference:                                                             
T = 800                                                                
qpi = Empirical(                                                       
    tf.get_variable(                                                   
    "qpi/params", [T, K],                                              
    initializer=tf.constant_initializer(1.0 / K)))                     
qmu = Empirical(tf.get_variable(                                       
    "qmu/params", [T, K, D],
    initializer=tf.zeros_initializer()))
qsigmasq = Empirical(tf.get_variable(                                  
    "qsigmasq/params", [T, K, D],
    initializer=tf.ones_initializer()))
qz = Empirical(tf.get_variable(                                        
    "qz/params", [T, N],
    initializer=tf.zeros_initializer(),
    dtype=tf.int32))

#Running Gibbs Sampling:
inference = ed.Gibbs({pi: qpi, mu: qmu, sigmasq: qsigmasq, z: qz},     
                     data={x: x_train})                                 
inference.initialize()                                                  

sess = ed.get_session()
tf.global_variables_initializer().run()

t_ph = tf.placeholder(tf.int32,[])                                      
running_cluster_means = tf.reduce_mean(qmu.params[:t_ph], 0)            

for _ in range(inference.n_iter):
  info_dict = inference.update()
  inference.print_progress(info_dict)
  t = info_dict['t']
  if t % inference.n_print == 0:
    print("\nInferred cluster means:")
    print(sess.run(running_cluster_means, {t_ph: t - 1}))                

## Criticism

###  Train: Sampling from posterior

In [ ]:
#Criticism:
# Calculate likelihood for each data point and cluster assignment,
# averaged over many posterior samples. `x_post` has shape (N, 100, K, D).
pi_sample = qpi.sample(100)
mu_sample = qmu.sample(100)                                                 
sigmasq_sample = qsigmasq.sample(100)                                       
x_post = Normal(loc=tf.ones([N, 1, 1, 1]) * mu_sample,                      
                scale=tf.ones([N, 1, 1, 1]) * tf.sqrt(sigmasq_sample))      
                                                                            
                                                                            
x_broadcasted = tf.tile(tf.reshape(x_train, [N, 1, 1, D]), [1, 100, K, 1])  
x_broadcasted = tf.cast(x_broadcasted, tf.float32)                          

### Train: Running log_liks

In [ ]:
#Log_liks:
log_liks = x_post.log_prob(x_broadcasted)                                   
log_liks = tf.reduce_sum(log_liks, 3)                                       
log_liks = tf.add(log_liks,tf.log(pi_sample))                               
log_liks = tf.subtract(tf.reduce_logsumexp(log_liks, 1),tf.log(100.0))      
log_dens=tf.reduce_logsumexp(log_liks,1)                                    

###  Train:  Samlet log_liks & cluster assignment

In [ ]:
tf.reduce_sum(log_dens).eval()                                

In [ ]:
#cluster with the highest likelihood for each data point assigned cluster value.
clusters = tf.argmax(log_liks, 1).eval()                    

### Test: Sampling

In [ ]:
#Prediction on test data
x_broadcasted = tf.tile(tf.reshape(x_test, [N, 1, 1, D]), [1, 100, K, 1]) 
x_broadcasted = tf.cast(x_broadcasted, tf.float32)                        

### Test: Log_liks

In [ ]:
#Log_liks:
log_liks = x_post.log_prob(x_broadcasted)                                   
log_liks = tf.reduce_sum(log_liks, 3)                                       
log_liks = tf.add(log_liks,tf.log(pi_sample))                               
log_liks = tf.subtract(tf.reduce_logsumexp(log_liks, 1),tf.log(100.0))      
log_dens=tf.reduce_logsumexp(log_liks,1)                                    

### Test: Samplet log_liks og cluster

In [ ]:
tf.reduce_sum(log_dens).eval()   

In [ ]:
#Cluster assignments:
clusters = tf.argmax(log_liks, 1).eval()     